In [ ]:
import os
Root = "./Ravdess_data"
os.chdir(Root)
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score


#Extract features (mfcc, chroma, mel) from a soundfile

In [ ]:

def extract_features(file_name, mfcc, chroma,mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result= np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
            result=np.hstack( (result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T, axis=0)
            result=np.hstack((result, mel))
    return result


# Emotions in the RAVDESS dataset

In [ ]:

emotions={
    '01':'neutral',
    '02':'calm',
    '03':'happy',
    '04':'sad',
    '05':'angry',
    '06':'fearful',
    '07':'disgust',
    '08':'surprised',
}

emotions to observe

In [ ]:
observed_emotions=['neutral', 'happy', 'fearful', 'sad']

Load the data and extract features for each sound file

In [ ]:
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("D:/Code/Ravdess_data/Actor_*/*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_features(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

Split the dataset

In [ ]:
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

In [ ]:
x_train

Get the shape of the training and testing datasets

In [ ]:
print((x_train.shape[0], x_test.shape[0]))

Get the number of features extracted

In [ ]:
print(f'features extracted: {x_train.shape[1]}')

Initialize the Multi Layer Perception Classifier

In [ ]:
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=600)

Train the model

In [ ]:
model.fit(x_train, y_train)

Predict for the set

In [ ]:
y_pred=model.predict(x_test)

In [ ]:
y_pred

Calculate the accuracy of our model

In [ ]:
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

Print the accuracy

In [ ]:
print("Accuracy: {:.2f}%".format(accuracy*100))

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
f1_score(y_test, y_pred,average=None)

In [ ]:
import pandas as pd
df=pd.DataFrame({'Actual': y_test, 'Predicted':y_pred})
df.head(20)

In [ ]:
import pickle
#Writing different model files to file
filename = 'modelForPrediction1'
pickle.dump(model,open(filename,'wb'))
    

loaded_model = pickle.load(open(filename, 'rb'))  #loading the model file from the storage

In [ ]:
feature=extract_features("./Ravdess_data/Actor_15/03-01-01-01-01-01-15.wav", mfcc=True, chroma=True, mel=True)

In [ ]:
feature=feature.reshape(1,-1)

In [ ]:
prediction=loaded_model.predict(feature)
print(prediction[0])

In [ ]:
feature